# RoPE

In order to let large language models (LLMs) know the order of the tokens, we use positional embeddings. There are absolute positional embedding and relative positional embeding. Today, we will talk about relative positional embedding with Rotary Position Embedding (RoPE) from [RoFormer: Enhanced Transformer with Rotary Position Embedding](https://arxiv.org/abs/2104.09864) by Su et al.

## Implementation

There are two ways to implement RoPE. One from [Llama](https://github.com/meta-llama/llama/blob/main/llama/model.py#L80) and another from [Hugging Face](https://github.com/huggingface/transformers/blob/e42587f596181396e1c4b63660abf0c736b10dae/src/transformers/models/llama/modeling_llama.py#L92).

In [ ]:
import torch
import torch.nn as nn
from typing import Optional, Tuple

In [ ]:
batch_size = 2
context_len = 3
num_heads = 4
head_dim = 16

In [ ]:
torch.manual_seed(123)
queries = torch.randn(batch_size, context_len, num_heads, head_dim)
keys = torch.randn(batch_size, context_len, num_heads, head_dim)
queries.shape, keys.shape

(torch.Size([2, 3, 4, 16]), torch.Size([2, 3, 4, 16]))

### Llama

Below is the code from [Llama 2 implementation](https://github.com/meta-llama/llama/blob/main/llama/model.py#L80). Let's try to understand it.

In [ ]:
def precompute_freqs_cis(dim: int, end: int, theta: float = 10000.0):
    """
    Precompute the frequency tensor for complex exponentials (cis) with given dimensions.

    This function calculates a frequency tensor with complex exponentials using the given dimension 'dim'
    and the end index 'end'. The 'theta' parameter scales the frequencies.
    The returned tensor contains complex values in complex64 data type.

    Args:
        dim (int): Dimension of the frequency tensor.
        end (int): End index for precomputing frequencies.
        theta (float, optional): Scaling factor for frequency computation. Defaults to 10000.0.

    Returns:
        torch.Tensor: Precomputed frequency tensor with complex exponentials.
    """
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
    t = torch.arange(end, device=freqs.device)  # type: ignore
    freqs = torch.outer(t, freqs).float()  # type: ignore
    freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    return freqs_cis

Let's go through code step by step. First, we need the arguments to pass.

In [ ]:
dim = head_dim
end = context_len
theta = 10_000

We want to create a frequency tensor. We are creating a `dim // 2` tensor because we get cos and sin from each frequency. And these are applied to each column as we will see. 

In [ ]:
torch.arange(0, dim, 2)[: (dim // 2)].float()

tensor([ 0.,  2.,  4.,  6.,  8., 10., 12., 14.])

Then we normalize by `dim`.

In [ ]:
(torch.arange(0, dim, 2)[: (dim // 2)].float() / dim)

tensor([0.0000, 0.1250, 0.2500, 0.3750, 0.5000, 0.6250, 0.7500, 0.8750])

In [ ]:
theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim)

tensor([1.0000e+00, 3.1623e+00, 1.0000e+01, 3.1623e+01, 1.0000e+02, 3.1623e+02,
        1.0000e+03, 3.1623e+03])

In [ ]:
1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))

tensor([1.0000e+00, 3.1623e-01, 1.0000e-01, 3.1623e-02, 1.0000e-02, 3.1623e-03,
        1.0000e-03, 3.1623e-04])

In [ ]:
freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
freqs

tensor([1.0000e+00, 3.1623e-01, 1.0000e-01, 3.1623e-02, 1.0000e-02, 3.1623e-03,
        1.0000e-03, 3.1623e-04])

- **High frequencies** (like 1.0) rotate quickly → capture short-range positional relationships
- **Low frequencies** (like 3.1623e-04 = 0.00031623) rotate slowly → capture long-range positional relationships

Think of it like a clock:
- A second hand (high frequency) completes many rotations quickly - good for distinguishing nearby moments
- An hour hand (low frequency) rotates slowly - good for distinguishing longer time periods

In RoPE, the high frequency dimensions help the model distinguish between adjacent tokens, while the low frequency dimensions help distinguish between tokens that are far apart in the sequence.

In [ ]:
t = torch.arange(end, device=freqs.device)
t

tensor([0, 1, 2])

In [ ]:
freqs = torch.outer(t, freqs).float()
freqs.shape, freqs

(torch.Size([3, 8]),
 tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.0000e+00, 3.1623e-01, 1.0000e-01, 3.1623e-02, 1.0000e-02, 3.1623e-03,
          1.0000e-03, 3.1623e-04],
         [2.0000e+00, 6.3246e-01, 2.0000e-01, 6.3246e-02, 2.0000e-02, 6.3246e-03,
          2.0000e-03, 6.3246e-04]]))

- **Lower dimension indices** → higher frequencies → faster rotation
- **Higher dimension indices** → lower frequencies → slower rotation

In [ ]:
freqs_cis = torch.polar(torch.ones_like(freqs), freqs)
freqs_cis.shape, freqs_cis

(torch.Size([3, 8]),
 tensor([[ 1.0000+0.0000e+00j,  1.0000+0.0000e+00j,  1.0000+0.0000e+00j,
           1.0000+0.0000e+00j,  1.0000+0.0000e+00j,  1.0000+0.0000e+00j,
           1.0000+0.0000e+00j,  1.0000+0.0000e+00j],
         [ 0.5403+8.4147e-01j,  0.9504+3.1098e-01j,  0.9950+9.9833e-02j,
           0.9995+3.1618e-02j,  0.9999+9.9998e-03j,  1.0000+3.1623e-03j,
           1.0000+1.0000e-03j,  1.0000+3.1623e-04j],
         [-0.4161+9.0930e-01j,  0.8066+5.9113e-01j,  0.9801+1.9867e-01j,
           0.9980+6.3203e-02j,  0.9998+1.9999e-02j,  1.0000+6.3245e-03j,
           1.0000+2.0000e-03j,  1.0000+6.3246e-04j]]))

![Figure 2: Long-term decay of RoPE](fig2_from_paper.png)

Image from the RoPE paper.

In [ ]:
def apply_rotary_emb(
    xq: torch.Tensor,
    xk: torch.Tensor,
    freqs_cis: torch.Tensor,
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Apply rotary embeddings to input tensors using the given frequency tensor.

    This function applies rotary embeddings to the given query 'xq' and key 'xk' tensors using the provided
    frequency tensor 'freqs_cis'. The input tensors are reshaped as complex numbers, and the frequency tensor
    is reshaped for broadcasting compatibility. The resulting tensors contain rotary embeddings and are
    returned as real tensors.

    Args:
        xq (torch.Tensor): Query tensor to apply rotary embeddings.
        xk (torch.Tensor): Key tensor to apply rotary embeddings.
        freqs_cis (torch.Tensor): Precomputed frequency tensor for complex exponentials.

    Returns:
        Tuple[torch.Tensor, torch.Tensor]: Tuple of modified query tensor and key tensor with rotary embeddings.
    """
    xq_ = torch.view_as_complex(xq.float().reshape(*xq.shape[:-1], -1, 2))
    xk_ = torch.view_as_complex(xk.float().reshape(*xk.shape[:-1], -1, 2))
    freqs_cis = reshape_for_broadcast(freqs_cis, xq_)
    xq_out = torch.view_as_real(xq_ * freqs_cis).flatten(3)
    xk_out = torch.view_as_real(xk_ * freqs_cis).flatten(3)
    return xq_out.type_as(xq), xk_out.type_as(xk)

queries and keys have shape `(batch_size, context_len, num_heads, head_dim)`.

In [ ]:
xq = queries
xk = keys

xq.shape, xk.shape, freqs_cis.shape

(torch.Size([2, 3, 4, 16]), torch.Size([2, 3, 4, 16]), torch.Size([3, 8]))

In [ ]:
xq.shape[:-1]

torch.Size([2, 3, 4])

Expand last dimension of query (head_dim), 16, into 2. One for cos, the other for sin.

In [ ]:
xq.float().reshape(*xq.shape[:-1], -1, 2).shape

torch.Size([2, 3, 4, 8, 2])

In [ ]:
xq[0,0,0]

tensor([ 0.3374, -0.1778, -0.3035, -0.5880,  0.3486,  0.6603, -0.2196, -0.3792,
         0.7671, -1.1925,  0.6984, -1.4097,  0.1794,  1.8951,  0.4954,  0.2692])

In [ ]:
xq_ = torch.view_as_complex(xq.float().reshape(*xq.shape[:-1], -1, 2))
xq_.shape

torch.Size([2, 3, 4, 8])

In [ ]:
xq_[0,0,0]

tensor([ 0.3374-0.1778j, -0.3035-0.5880j,  0.3486+0.6603j, -0.2196-0.3792j,
         0.7671-1.1925j,  0.6984-1.4097j,  0.1794+1.8951j,  0.4954+0.2692j])

Then, `reshape_for_broadcast` happens. 

In [ ]:
def reshape_for_broadcast(freqs_cis: torch.Tensor, x: torch.Tensor):
    """
    Reshape frequency tensor for broadcasting it with another tensor.

    This function reshapes the frequency tensor to have the same shape as the target tensor 'x'
    for the purpose of broadcasting the frequency tensor during element-wise operations.

    Args:
        freqs_cis (torch.Tensor): Frequency tensor to be reshaped.
        x (torch.Tensor): Target tensor for broadcasting compatibility.

    Returns:
        torch.Tensor: Reshaped frequency tensor.

    Raises:
        AssertionError: If the frequency tensor doesn't match the expected shape.
        AssertionError: If the target tensor 'x' doesn't have the expected number of dimensions.
    """
    ndim = x.ndim
    assert 0 <= 1 < ndim
    assert freqs_cis.shape == (x.shape[1], x.shape[-1])
    shape = [d if i == 1 or i == ndim - 1 else 1 for i, d in enumerate(x.shape)]
    return freqs_cis.view(*shape)

Let's remind ourselves of the shapes. 

In [ ]:
freqs_cis.shape, queries.shape

(torch.Size([3, 8]), torch.Size([2, 3, 4, 16]))

Rembmer `queries` and `keys` have shape: `(batch_size, context_len, num_heads, head_dim)`. `reshape_for_broadcast` is used after queries and keys are transformed into complex space according to `apply_rotary_emb`. We want `freqs_cis` to be `(1, context_len, 1, head_dim)`. Let's use `queries` as `x`.

In [ ]:
x = xq_
x.shape

torch.Size([2, 3, 4, 8])

In [ ]:
ndim = x.ndim
ndim

4

In [ ]:
assert 0 <= 1 < ndim
assert freqs_cis.shape == (x.shape[1], x.shape[-1])

In [ ]:
shape = [d if i == 1 or i == ndim - 1 else 1 for i, d in enumerate(x.shape)]
shape

[1, 3, 1, 8]

Going back to `apply_rotary_emb`.

In [ ]:
freqs_cis = reshape_for_broadcast(freqs_cis, xq_)
freqs_cis.shape

torch.Size([1, 3, 1, 8])

In [ ]:
(xq_ * freqs_cis).shape

torch.Size([2, 3, 4, 8])

In [ ]:
xq_[0,1,0]

tensor([ 0.5146+0.9938j, -0.2587-1.0826j, -0.0444+1.6236j, -2.3229+1.0878j,
         0.6716+0.6933j, -0.9487-0.0765j, -0.1526+0.1167j,  0.4403-1.4465j])

In [ ]:
freqs_cis[0,1,0]

tensor([0.5403+8.4147e-01j, 0.9504+3.1098e-01j, 0.9950+9.9833e-02j,
        0.9995+3.1618e-02j, 0.9999+9.9998e-03j, 1.0000+3.1623e-03j,
        1.0000+1.0000e-03j, 1.0000+3.1623e-04j])

In [ ]:
(xq_ * freqs_cis)[0,1,0]

tensor([-0.5582+0.9700j,  0.0908-1.1093j, -0.2062+1.6110j, -2.3561+1.0138j,
         0.6646+0.7000j, -0.9485-0.0795j, -0.1528+0.1166j,  0.4407-1.4464j])

In [ ]:
# Under the hood
0.5146*0.5403-0.9938*8.4147e-01, 0.5146*8.4147e-01+0.9938*0.5403

(-0.5582145060000001, 0.969970602)

The multiplication `xq_ * freqs_cis` is complex number multiplication, which performs the rotation.

When you multiply two complex numbers:
```
(a + bi) * (cos(θ) + i*sin(θ)) = (a*cos(θ) - b*sin(θ)) + i*(a*sin(θ) + b*cos(θ))
```

This is exactly the 2D rotation formula! So each complex number in `xq_` gets rotated by the corresponding angle in `freqs_cis`.

Each position gets its own rotation angles, and each dimension pair gets its own frequency, but all heads share the same positional encoding.

This is what the transformation looks like.

In [ ]:
(xq_ * freqs_cis)[0,0]

tensor([[ 0.3374-0.1778j, -0.3035-0.5880j,  0.3486+0.6603j, -0.2196-0.3792j,
          0.7671-1.1925j,  0.6984-1.4097j,  0.1794+1.8951j,  0.4954+0.2692j],
        [-0.0770-1.0205j, -0.1690+0.9178j,  1.5810+1.3010j,  1.2753-0.2010j,
          0.4965-1.5723j,  0.9666-1.1481j, -1.1589+0.3255j, -0.6315-2.8400j],
        [-1.3250+0.1784j, -2.1338+1.0524j, -0.3885-0.9343j, -0.4991-1.0867j,
          0.8805+1.5542j,  0.6266-0.1755j,  0.0983-0.0935j,  0.2662-0.5850j],
        [ 0.8768+1.6221j, -1.4779+1.1331j, -1.2203+1.3139j,  1.0533+0.1388j,
          2.2473-0.8036j, -0.2808+0.7697j, -0.6596-0.7979j,  0.1838+0.2293j]])

![Figure 1 from Paper](fig1_from_paper.png)

Image from the RoPE paper.

When we use `torch.view_as_real`, 

In [ ]:
torch.view_as_real(xq_ * freqs_cis)[0,0]

tensor([[[ 0.3374, -0.1778],
         [-0.3035, -0.5880],
         [ 0.3486,  0.6603],
         [-0.2196, -0.3792],
         [ 0.7671, -1.1925],
         [ 0.6984, -1.4097],
         [ 0.1794,  1.8951],
         [ 0.4954,  0.2692]],

        [[-0.0770, -1.0205],
         [-0.1690,  0.9178],
         [ 1.5810,  1.3010],
         [ 1.2753, -0.2010],
         [ 0.4965, -1.5723],
         [ 0.9666, -1.1481],
         [-1.1589,  0.3255],
         [-0.6315, -2.8400]],

        [[-1.3250,  0.1784],
         [-2.1338,  1.0524],
         [-0.3885, -0.9343],
         [-0.4991, -1.0867],
         [ 0.8805,  1.5542],
         [ 0.6266, -0.1755],
         [ 0.0983, -0.0935],
         [ 0.2662, -0.5850]],

        [[ 0.8768,  1.6221],
         [-1.4779,  1.1331],
         [-1.2203,  1.3139],
         [ 1.0533,  0.1388],
         [ 2.2473, -0.8036],
         [-0.2808,  0.7697],
         [-0.6596, -0.7979],
         [ 0.1838,  0.2293]]])

In [ ]:
xq_out[0,0]

tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  0.3486,  0.6603, -0.2196, -0.3792,
          0.7671, -1.1925,  0.6984, -1.4097,  0.1794,  1.8951,  0.4954,  0.2692],
        [-0.0770, -1.0205, -0.1690,  0.9178,  1.5810,  1.3010,  1.2753, -0.2010,
          0.4965, -1.5723,  0.9666, -1.1481, -1.1589,  0.3255, -0.6315, -2.8400],
        [-1.3250,  0.1784, -2.1338,  1.0524, -0.3885, -0.9343, -0.4991, -1.0867,
          0.8805,  1.5542,  0.6266, -0.1755,  0.0983, -0.0935,  0.2662, -0.5850],
        [ 0.8768,  1.6221, -1.4779,  1.1331, -1.2203,  1.3139,  1.0533,  0.1388,
          2.2473, -0.8036, -0.2808,  0.7697, -0.6596, -0.7979,  0.1838,  0.2293]])

In [ ]:
xq_out = torch.view_as_real(xq_ * freqs_cis).flatten(3)
xq_out.shape

torch.Size([2, 3, 4, 16])

### torchtune implementation

torchtune's [RotaryPositionalEmbeddings](https://docs.pytorch.org/torchtune/stable/generated/torchtune.modules.RotaryPositionalEmbeddings.html) follows llama's implementation.

In [ ]:
from torchtune.modules import RotaryPositionalEmbeddings

In [ ]:
tt_rope_emb = RotaryPositionalEmbeddings(dim=head_dim, max_seq_len=context_len, base=theta)
tt_rope_emb

RotaryPositionalEmbeddings()

In [ ]:
tt_queries_rot = tt_rope_emb(queries)
tt_queries_rot.shape

torch.Size([2, 3, 4, 16])

This gives us the same result as llama implementation.

In [ ]:
assert torch.allclose(tt_queries_rot, xq_out)

## LitGPT implementation

In [ ]:
# LitGPT code function `litgpt_build_rope_cache` from https://github.com/Lightning-AI/litgpt/blob/main/litgpt/model.py
# LitGPT is licensed under Apache v2: https://github.com/Lightning-AI/litgpt/blob/main/LICENSE
def litgpt_build_rope_cache(
    seq_len: int,
    n_elem: int,
    device: Optional[torch.device] = None,
    base: int = 10000,
    condense_ratio: int = 1,
    extra_config: Optional[dict] = None,
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Enhanced Transformer with Rotary Position Embedding.

    Args:
        seq_len (int): Sequence length.
        n_elem (int): Number of elements (head dimension).
        device (torch.device, optional): Device for tensor allocations.
        base (int, optional): Base for computing inverse frequencies.
        condense_ratio (int, optional): Ratio to condense the position indices.
        extra_config (dict, optional): Configuration parameters for frequency adjustments (used by Llama 3.1 and 3.2)

    Returns:
        Tuple[torch.Tensor, torch.Tensor]: Cosine and sine caches for RoPE.
    """
    # Compute the inverse frequencies theta
    theta = 1.0 / (base ** (torch.arange(0, n_elem, 2, device=device).float() / n_elem))

    if extra_config is not None:
        orig_context_len = extra_config["original_max_seq_len"]
        factor = extra_config["factor"]
        low_freq_factor = extra_config["low_freq_factor"]
        high_freq_factor = extra_config["high_freq_factor"]

        wavelen = 2 * torch.pi / theta
        ratio = orig_context_len / wavelen
        smooth_factor = (ratio - low_freq_factor) / (high_freq_factor - low_freq_factor)
        smooth_factor = torch.clamp(smooth_factor, min=0.0, max=1.0)

        # Compute adjusted_theta without masked indexing
        adjusted_theta = (1 - smooth_factor) * (theta / factor) + smooth_factor * theta
        theta = adjusted_theta

    # Create position indices `[0, 1, ..., seq_len - 1]`
    seq_idx = torch.arange(seq_len, device=device) / condense_ratio

    # Calculate the product of position index and $\theta_i$
    idx_theta = torch.outer(seq_idx, theta).repeat(1, 2)

    return torch.cos(idx_theta), torch.sin(idx_theta)

In [ ]:
litgpt_cos, litgpt_sin = litgpt_build_rope_cache(context_len, n_elem=head_dim, base=theta)
litgpt_cos.shape, litgpt_sin.shape

(torch.Size([3, 16]), torch.Size([3, 16]))

In [ ]:
freqs_cis = precompute_freqs_cis(dim=head_dim, end=context_len, theta=theta)
freqs_cis.shape

torch.Size([3, 8])

In [ ]:
litgpt_cos[1], litgpt_sin[1]

(tensor([0.5403, 0.9504, 0.9950, 0.9995, 0.9999, 1.0000, 1.0000, 1.0000, 0.5403,
         0.9504, 0.9950, 0.9995, 0.9999, 1.0000, 1.0000, 1.0000]),
 tensor([8.4147e-01, 3.1098e-01, 9.9833e-02, 3.1618e-02, 9.9998e-03, 3.1623e-03,
         1.0000e-03, 3.1623e-04, 8.4147e-01, 3.1098e-01, 9.9833e-02, 3.1618e-02,
         9.9998e-03, 3.1623e-03, 1.0000e-03, 3.1623e-04]))

In [ ]:
freqs_cis[1]

tensor([0.5403+8.4147e-01j, 0.9504+3.1098e-01j, 0.9950+9.9833e-02j,
        0.9995+3.1618e-02j, 0.9999+9.9998e-03j, 1.0000+3.1623e-03j,
        1.0000+1.0000e-03j, 1.0000+3.1623e-04j])

In [ ]:
torch.cos(freqs)[1]

tensor([0.5403, 0.9504, 0.9950, 0.9995, 0.9999, 1.0000, 1.0000, 1.0000])

In [ ]:
torch.cos(freqs_cis)[1]

tensor([1.1795-4.8577e-01j, 0.6097-2.5713e-01j, 0.5472-8.3876e-02j,
        0.5410-2.6601e-02j, 0.5404-8.4144e-03j, 0.5403-2.6610e-03j,
        0.5403-8.4147e-04j, 0.5403-2.6610e-04j])

In [ ]:
torch.stack([litgpt_cos, litgpt_sin], dim=-1).shape

torch.Size([3, 16, 2])

In [ ]:
torch.view_as_real(freqs_cis).shape, torch.view_as_real(freqs_cis)[1]

(torch.Size([3, 8, 2]),
 tensor([[5.4030e-01, 8.4147e-01],
         [9.5042e-01, 3.1098e-01],
         [9.9500e-01, 9.9833e-02],
         [9.9950e-01, 3.1618e-02],
         [9.9995e-01, 9.9998e-03],
         [9.9999e-01, 3.1623e-03],
         [1.0000e+00, 1.0000e-03],
         [1.0000e+00, 3.1623e-04]]))

In [ ]:
torch.view_as_real(freqs_cis)[..., 0][1]

tensor([0.5403, 0.9504, 0.9950, 0.9995, 0.9999, 1.0000, 1.0000, 1.0000])

In [ ]:
assert torch.allclose(litgpt_cos, torch.view_as_real(freqs_cis)[..., 0].repeat(1,2))

In [ ]:
# LitGPT code from https://github.com/Lightning-AI/litgpt/blob/main/litgpt/model.py
# LitGPT is licensed under Apache v2: https://github.com/Lightning-AI/litgpt/blob/main/LICENSE
def litgpt_apply_rope(x: torch.Tensor, cos: torch.Tensor, sin: torch.Tensor) -> torch.Tensor:
    head_size = x.size(-1)
    a = x[..., : head_size // 2]  # (B, nh, T, hs/2)
    b = x[..., head_size // 2:]  # (B, nh, T, hs/2)
    rotated = torch.cat((-b, a), dim=-1)  # (B, nh, T, hs)
    if cos.dim() > 1:
        # batch dimensions must align
        # sin/cos are (B, T, hs) so we unsqeeze -3 for nh
        # we count from back because all of apply_rope does
        cos = cos.unsqueeze(-3)
        sin = sin.unsqueeze(-3)

    roped = (x * cos) + (rotated * sin)
    return roped.to(dtype=x.dtype)

`(a + bi) * (cos(θ) + i*sin(θ)) = (acos(θ) - bsin(θ)) + i(asin(θ) + bcos(θ))`

`(acos(θ) - bsin(θ)) + i(asin(θ) + bcos(θ)) = (acos(θ)+ibcos(θ)) + (-bsin(θ)+iasin(θ))`

In [ ]:
queries.shape

torch.Size([2, 3, 4, 16])

In [ ]:
assert torch.allclose(tt_queries_rot, freqs_cis)

## Hugging Face implementation

Hugging Face [modeling_llama](https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/modeling_llama.py).

In [ ]:
queires_t = queries.transpose(1,2)
keys_t = keys.transpose(1,2)
queires_t.shape, keys_t.shape

(torch.Size([2, 4, 3, 16]), torch.Size([2, 4, 3, 16]))

In [ ]:
queries

tensor([[[[ 3.3737e-01, -1.7778e-01, -3.0353e-01, -5.8801e-01,  3.4861e-01,
            6.6034e-01, -2.1964e-01, -3.7917e-01,  7.6711e-01, -1.1925e+00,
            6.9835e-01, -1.4097e+00,  1.7938e-01,  1.8951e+00,  4.9545e-01,
            2.6920e-01],
          [-7.7020e-02, -1.0205e+00, -1.6896e-01,  9.1776e-01,  1.5810e+00,
            1.3010e+00,  1.2753e+00, -2.0095e-01,  4.9647e-01, -1.5723e+00,
            9.6657e-01, -1.1481e+00, -1.1589e+00,  3.2547e-01, -6.3151e-01,
           -2.8400e+00],
          [-1.3250e+00,  1.7843e-01, -2.1338e+00,  1.0524e+00, -3.8848e-01,
           -9.3435e-01, -4.9914e-01, -1.0867e+00,  8.8054e-01,  1.5542e+00,
            6.2662e-01, -1.7549e-01,  9.8284e-02, -9.3507e-02,  2.6621e-01,
           -5.8504e-01],
          [ 8.7684e-01,  1.6221e+00, -1.4779e+00,  1.1331e+00, -1.2203e+00,
            1.3139e+00,  1.0533e+00,  1.3881e-01,  2.2473e+00, -8.0364e-01,
           -2.8084e-01,  7.6968e-01, -6.5956e-01, -7.9793e-01,  1.8383e-01,
            2

In [ ]:
def permute(w, n_heads, dim1=dim, dim2=dim):
    return w.view(n_heads, dim1 // n_heads // 2, 2, dim2).transpose(1, 2).reshape(dim1, dim2)

In [ ]:
queries.shape

torch.Size([2, 3, 4, 16])

In [ ]:
queries[0,0]

tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  0.3486,  0.6603, -0.2196, -0.3792,
          0.7671, -1.1925,  0.6984, -1.4097,  0.1794,  1.8951,  0.4954,  0.2692],
        [-0.0770, -1.0205, -0.1690,  0.9178,  1.5810,  1.3010,  1.2753, -0.2010,
          0.4965, -1.5723,  0.9666, -1.1481, -1.1589,  0.3255, -0.6315, -2.8400],
        [-1.3250,  0.1784, -2.1338,  1.0524, -0.3885, -0.9343, -0.4991, -1.0867,
          0.8805,  1.5542,  0.6266, -0.1755,  0.0983, -0.0935,  0.2662, -0.5850],
        [ 0.8768,  1.6221, -1.4779,  1.1331, -1.2203,  1.3139,  1.0533,  0.1388,
          2.2473, -0.8036, -0.2808,  0.7697, -0.6596, -0.7979,  0.1838,  0.2293]])

In [ ]:
queries.transpose(2,3).shape

torch.Size([2, 3, 16, 4])

In [ ]:
queries.transpose(2,3)[0,0]

tensor([[ 0.3374, -0.0770, -1.3250,  0.8768],
        [-0.1778, -1.0205,  0.1784,  1.6221],
        [-0.3035, -0.1690, -2.1338, -1.4779],
        [-0.5880,  0.9178,  1.0524,  1.1331],
        [ 0.3486,  1.5810, -0.3885, -1.2203],
        [ 0.6603,  1.3010, -0.9343,  1.3139],
        [-0.2196,  1.2753, -0.4991,  1.0533],
        [-0.3792, -0.2010, -1.0867,  0.1388],
        [ 0.7671,  0.4965,  0.8805,  2.2473],
        [-1.1925, -1.5723,  1.5542, -0.8036],
        [ 0.6984,  0.9666,  0.6266, -0.2808],
        [-1.4097, -1.1481, -0.1755,  0.7697],
        [ 0.1794, -1.1589,  0.0983, -0.6596],
        [ 1.8951,  0.3255, -0.0935, -0.7979],
        [ 0.4954, -0.6315,  0.2662,  0.1838],
        [ 0.2692, -2.8400, -0.5850,  0.2293]])

In [ ]:
queries.numel()

384

In [ ]:
queries.transpose(2,3).view(batch_size, context_len, num_heads, 16 // num_heads // 2, 2, 4).shape

torch.Size([2, 3, 4, 2, 2, 4])

In [ ]:
queries.transpose(2,3).view(batch_size, context_len, num_heads, 16 // num_heads // 2, 2, 4).transpose(-2, -3).shape

torch.Size([2, 3, 4, 2, 2, 4])

In [ ]:
(queries.transpose(2,3)
 .view(batch_size, context_len, num_heads, 16 // num_heads // 2, 2, 4)
 .transpose(-2, -3)
 .reshape(batch_size, context_len,16, 4)).shape

torch.Size([2, 3, 16, 4])

In [ ]:
(queries.transpose(2,3)
 .view(batch_size, context_len, num_heads, 16 // num_heads // 2, 2, 4)
 .transpose(-2, -3)
 .reshape(batch_size, context_len,16, 4)).transpose(2,3)[0,0]

tensor([[ 0.3374, -0.3035, -0.1778, -0.5880,  0.3486, -0.2196,  0.6603, -0.3792,
          0.7671,  0.6984, -1.1925, -1.4097,  0.1794,  0.4954,  1.8951,  0.2692],
        [-0.0770, -0.1690, -1.0205,  0.9178,  1.5810,  1.2753,  1.3010, -0.2010,
          0.4965,  0.9666, -1.5723, -1.1481, -1.1589, -0.6315,  0.3255, -2.8400],
        [-1.3250, -2.1338,  0.1784,  1.0524, -0.3885, -0.4991, -0.9343, -1.0867,
          0.8805,  0.6266,  1.5542, -0.1755,  0.0983,  0.2662, -0.0935, -0.5850],
        [ 0.8768, -1.4779,  1.6221,  1.1331, -1.2203,  1.0533,  1.3139,  0.1388,
          2.2473, -0.2808, -0.8036,  0.7697, -0.6596,  0.1838, -0.7979,  0.2293]])

In [ ]:
permute(queries.transpose(2,3), num_heads, 16, 4).shape

RuntimeError: shape '[4, 2, 2, 4]' is invalid for input of size 384

In [ ]:
import transformers
from transformers.models.llama.modeling_llama import LlamaRotaryEmbedding, apply_rotary_pos_emb

In [ ]:
transformers_version = transformers.__version__
transformers_version

'4.50.1'

In [ ]:
class RoPEConfig:
    dim: int = head_dim
    rope_theta = theta
    max_position_embeddings: int = context_len
    hidden_size = head_dim * num_heads
    num_attention_heads = num_heads

config = RoPEConfig()
hf_rot_emb = LlamaRotaryEmbedding(config=config)

In [ ]:
position_ids = torch.arange(context_len, dtype=torch.long).unsqueeze(0)
position_ids

tensor([[0, 1, 2]])

In [ ]:
hf_cos, hf_sin = hf_rot_emb(queries, position_ids)
hf_queries_t_rot, hf_keys_t_rot = apply_rotary_pos_emb(queires_t, keys_t, hf_cos, hf_sin)
hf_queries_t_rot.shape, hf_keys_t_rot.shape

(torch.Size([2, 4, 3, 16]), torch.Size([2, 4, 3, 16]))

In [ ]:
hf_queries_rot = hf_queries_t_rot.transpose(1,2)
hf_keys_rot = hf_keys_t_rot.transpose(1,2)
hf_queries_rot.shape, hf_keys_rot.shape

(torch.Size([2, 3, 4, 16]), torch.Size([2, 3, 4, 16]))

In [ ]:
assert torch.allclose(tt_queries_rot, permute(hf_queries_rot, n_heads=num_heads))

RuntimeError: shape '[4, 2, 2, 16]' is invalid for input of size 384

In [ ]:
torch.testing.assert_close(sin.repeat(1,2), ref_sin.squeeze(0))
torch.testing.assert_close(cos.repeat(1,2), ref_cos.squeeze(0))
torch.testing.assert_close(keys_rot, ref_keys_rot)
torch.testing.assert_close(queries_rot, ref_queries_rot)

In [ ]:
def permute(w, n_heads, dim1=dim, dim2=dim):
    return w.view(n_heads, dim1 // n_heads // 2, 2, dim2).transpose(1, 2).reshape(dim1, dim2)

For HF implementation to match with torchtune, we have to permute this.

In [ ]:
dim=8
t = torch.arange(8*8).reshape(dim,dim)
t

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7],
        [ 8,  9, 10, 11, 12, 13, 14, 15],
        [16, 17, 18, 19, 20, 21, 22, 23],
        [24, 25, 26, 27, 28, 29, 30, 31],
        [32, 33, 34, 35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44, 45, 46, 47],
        [48, 49, 50, 51, 52, 53, 54, 55],
        [56, 57, 58, 59, 60, 61, 62, 63]])

In [ ]:
n_heads=2
t_ = t.view(n_heads, dim//n_heads//2, 2, dim)
t_.shape, t_

(torch.Size([2, 2, 2, 8]),
 tensor([[[[ 0,  1,  2,  3,  4,  5,  6,  7],
           [ 8,  9, 10, 11, 12, 13, 14, 15]],
 
          [[16, 17, 18, 19, 20, 21, 22, 23],
           [24, 25, 26, 27, 28, 29, 30, 31]]],
 
 
         [[[32, 33, 34, 35, 36, 37, 38, 39],
           [40, 41, 42, 43, 44, 45, 46, 47]],
 
          [[48, 49, 50, 51, 52, 53, 54, 55],
           [56, 57, 58, 59, 60, 61, 62, 63]]]]))

In [ ]:
t_t = t_.transpose(1,2)
t_t.transpose(1,2).shape, t_t

(torch.Size([2, 2, 2, 8]),
 tensor([[[[ 0,  1,  2,  3,  4,  5,  6,  7],
           [16, 17, 18, 19, 20, 21, 22, 23]],
 
          [[ 8,  9, 10, 11, 12, 13, 14, 15],
           [24, 25, 26, 27, 28, 29, 30, 31]]],
 
 
         [[[32, 33, 34, 35, 36, 37, 38, 39],
           [48, 49, 50, 51, 52, 53, 54, 55]],
 
          [[40, 41, 42, 43, 44, 45, 46, 47],
           [56, 57, 58, 59, 60, 61, 62, 63]]]]))

In [ ]:
t_t.reshape(dim,dim)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7],
        [16, 17, 18, 19, 20, 21, 22, 23],
        [ 8,  9, 10, 11, 12, 13, 14, 15],
        [24, 25, 26, 27, 28, 29, 30, 31],
        [32, 33, 34, 35, 36, 37, 38, 39],
        [48, 49, 50, 51, 52, 53, 54, 55],
        [40, 41, 42, 43, 44, 45, 46, 47],
        [56, 57, 58, 59, 60, 61, 62, 63]])

Permuting on columns.

In [ ]:
t.T

tensor([[ 0,  8, 16, 24, 32, 40, 48, 56],
        [ 1,  9, 17, 25, 33, 41, 49, 57],
        [ 2, 10, 18, 26, 34, 42, 50, 58],
        [ 3, 11, 19, 27, 35, 43, 51, 59],
        [ 4, 12, 20, 28, 36, 44, 52, 60],
        [ 5, 13, 21, 29, 37, 45, 53, 61],
        [ 6, 14, 22, 30, 38, 46, 54, 62],
        [ 7, 15, 23, 31, 39, 47, 55, 63]])

In [ ]:
permute(t.T, n_heads, dim, dim)

tensor([[ 0,  8, 16, 24, 32, 40, 48, 56],
        [ 2, 10, 18, 26, 34, 42, 50, 58],
        [ 1,  9, 17, 25, 33, 41, 49, 57],
        [ 3, 11, 19, 27, 35, 43, 51, 59],
        [ 4, 12, 20, 28, 36, 44, 52, 60],
        [ 6, 14, 22, 30, 38, 46, 54, 62],
        [ 5, 13, 21, 29, 37, 45, 53, 61],
        [ 7, 15, 23, 31, 39, 47, 55, 63]])

In [ ]:
permute(t.T, n_heads, dim, dim).T

tensor([[ 0,  2,  1,  3,  4,  6,  5,  7],
        [ 8, 10,  9, 11, 12, 14, 13, 15],
        [16, 18, 17, 19, 20, 22, 21, 23],
        [24, 26, 25, 27, 28, 30, 29, 31],
        [32, 34, 33, 35, 36, 38, 37, 39],
        [40, 42, 41, 43, 44, 46, 45, 47],
        [48, 50, 49, 51, 52, 54, 53, 55],
        [56, 58, 57, 59, 60, 62, 61, 63]])

Let's add a batch dimension.

In [ ]:
dim=8
bs=2
t = torch.arange(dim*dim*bs).reshape(bs,dim,dim)
t

tensor([[[  0,   1,   2,   3,   4,   5,   6,   7],
         [  8,   9,  10,  11,  12,  13,  14,  15],
         [ 16,  17,  18,  19,  20,  21,  22,  23],
         [ 24,  25,  26,  27,  28,  29,  30,  31],
         [ 32,  33,  34,  35,  36,  37,  38,  39],
         [ 40,  41,  42,  43,  44,  45,  46,  47],
         [ 48,  49,  50,  51,  52,  53,  54,  55],
         [ 56,  57,  58,  59,  60,  61,  62,  63]],

        [[ 64,  65,  66,  67,  68,  69,  70,  71],
         [ 72,  73,  74,  75,  76,  77,  78,  79],
         [ 80,  81,  82,  83,  84,  85,  86,  87],
         [ 88,  89,  90,  91,  92,  93,  94,  95],
         [ 96,  97,  98,  99, 100, 101, 102, 103],
         [104, 105, 106, 107, 108, 109, 110, 111],
         [112, 113, 114, 115, 116, 117, 118, 119],
         [120, 121, 122, 123, 124, 125, 126, 127]]])

In [ ]:
n_heads=2
t_ = t.view(bs, n_heads, dim//n_heads//2, 2, dim)
t_.shape, t_

(torch.Size([2, 2, 2, 2, 8]),
 tensor([[[[[  0,   1,   2,   3,   4,   5,   6,   7],
            [  8,   9,  10,  11,  12,  13,  14,  15]],
 
           [[ 16,  17,  18,  19,  20,  21,  22,  23],
            [ 24,  25,  26,  27,  28,  29,  30,  31]]],
 
 
          [[[ 32,  33,  34,  35,  36,  37,  38,  39],
            [ 40,  41,  42,  43,  44,  45,  46,  47]],
 
           [[ 48,  49,  50,  51,  52,  53,  54,  55],
            [ 56,  57,  58,  59,  60,  61,  62,  63]]]],
 
 
 
         [[[[ 64,  65,  66,  67,  68,  69,  70,  71],
            [ 72,  73,  74,  75,  76,  77,  78,  79]],
 
           [[ 80,  81,  82,  83,  84,  85,  86,  87],
            [ 88,  89,  90,  91,  92,  93,  94,  95]]],
 
 
          [[[ 96,  97,  98,  99, 100, 101, 102, 103],
            [104, 105, 106, 107, 108, 109, 110, 111]],
 
           [[112, 113, 114, 115, 116, 117, 118, 119],
            [120, 121, 122, 123, 124, 125, 126, 127]]]]]))

In [ ]:
t_t = t_.transpose(1,2)
t_t.transpose(1,2).shape, t_t

(torch.Size([2, 2, 2, 2, 8]),
 tensor([[[[[  0,   1,   2,   3,   4,   5,   6,   7],
            [  8,   9,  10,  11,  12,  13,  14,  15]],
 
           [[ 32,  33,  34,  35,  36,  37,  38,  39],
            [ 40,  41,  42,  43,  44,  45,  46,  47]]],
 
 
          [[[ 16,  17,  18,  19,  20,  21,  22,  23],
            [ 24,  25,  26,  27,  28,  29,  30,  31]],
 
           [[ 48,  49,  50,  51,  52,  53,  54,  55],
            [ 56,  57,  58,  59,  60,  61,  62,  63]]]],
 
 
 
         [[[[ 64,  65,  66,  67,  68,  69,  70,  71],
            [ 72,  73,  74,  75,  76,  77,  78,  79]],
 
           [[ 96,  97,  98,  99, 100, 101, 102, 103],
            [104, 105, 106, 107, 108, 109, 110, 111]]],
 
 
          [[[ 80,  81,  82,  83,  84,  85,  86,  87],
            [ 88,  89,  90,  91,  92,  93,  94,  95]],
 
           [[112, 113, 114, 115, 116, 117, 118, 119],
            [120, 121, 122, 123, 124, 125, 126, 127]]]]]))

In [ ]:
t_t.reshape(bs,dim,dim)

tensor([[[  0,   1,   2,   3,   4,   5,   6,   7],
         [  8,   9,  10,  11,  12,  13,  14,  15],
         [ 32,  33,  34,  35,  36,  37,  38,  39],
         [ 40,  41,  42,  43,  44,  45,  46,  47],
         [ 16,  17,  18,  19,  20,  21,  22,  23],
         [ 24,  25,  26,  27,  28,  29,  30,  31],
         [ 48,  49,  50,  51,  52,  53,  54,  55],
         [ 56,  57,  58,  59,  60,  61,  62,  63]],

        [[ 64,  65,  66,  67,  68,  69,  70,  71],
         [ 72,  73,  74,  75,  76,  77,  78,  79],
         [ 96,  97,  98,  99, 100, 101, 102, 103],
         [104, 105, 106, 107, 108, 109, 110, 111],
         [ 80,  81,  82,  83,  84,  85,  86,  87],
         [ 88,  89,  90,  91,  92,  93,  94,  95],
         [112, 113, 114, 115, 116, 117, 118, 119],
         [120, 121, 122, 123, 124, 125, 126, 127]]])

Permuting on columns.

In [ ]:
t.transpose(1,2)

tensor([[[  0,   8,  16,  24,  32,  40,  48,  56],
         [  1,   9,  17,  25,  33,  41,  49,  57],
         [  2,  10,  18,  26,  34,  42,  50,  58],
         [  3,  11,  19,  27,  35,  43,  51,  59],
         [  4,  12,  20,  28,  36,  44,  52,  60],
         [  5,  13,  21,  29,  37,  45,  53,  61],
         [  6,  14,  22,  30,  38,  46,  54,  62],
         [  7,  15,  23,  31,  39,  47,  55,  63]],

        [[ 64,  72,  80,  88,  96, 104, 112, 120],
         [ 65,  73,  81,  89,  97, 105, 113, 121],
         [ 66,  74,  82,  90,  98, 106, 114, 122],
         [ 67,  75,  83,  91,  99, 107, 115, 123],
         [ 68,  76,  84,  92, 100, 108, 116, 124],
         [ 69,  77,  85,  93, 101, 109, 117, 125],
         [ 70,  78,  86,  94, 102, 110, 118, 126],
         [ 71,  79,  87,  95, 103, 111, 119, 127]]])

In [ ]:
permute(t.T, n_heads, dim, dim)

tensor([[ 0,  8, 16, 24, 32, 40, 48, 56],
        [ 2, 10, 18, 26, 34, 42, 50, 58],
        [ 1,  9, 17, 25, 33, 41, 49, 57],
        [ 3, 11, 19, 27, 35, 43, 51, 59],
        [ 4, 12, 20, 28, 36, 44, 52, 60],
        [ 6, 14, 22, 30, 38, 46, 54, 62],
        [ 5, 13, 21, 29, 37, 45, 53, 61],
        [ 7, 15, 23, 31, 39, 47, 55, 63]])

In [ ]:
permute(t.T, n_heads, dim, dim).T

tensor([[ 0,  2,  1,  3,  4,  6,  5,  7],
        [ 8, 10,  9, 11, 12, 14, 13, 15],
        [16, 18, 17, 19, 20, 22, 21, 23],
        [24, 26, 25, 27, 28, 30, 29, 31],
        [32, 34, 33, 35, 36, 38, 37, 39],
        [40, 42, 41, 43, 44, 46, 45, 47],
        [48, 50, 49, 51, 52, 54, 53, 55],
        [56, 58, 57, 59, 60, 62, 61, 63]])

In [ ]:
permute(t, 2, dim, dim)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7],
        [16, 17, 18, 19, 20, 21, 22, 23],
        [ 8,  9, 10, 11, 12, 13, 14, 15],
        [24, 25, 26, 27, 28, 29, 30, 31],
        [32, 33, 34, 35, 36, 37, 38, 39],
        [48, 49, 50, 51, 52, 53, 54, 55],
        [40, 41, 42, 43, 44, 45, 46, 47],
        [56, 57, 58, 59, 60, 61, 62, 63]])